<a href="https://colab.research.google.com/github/ayobamiakomolafe/UmojaHack-Africa-2021-Zindi/blob/main/UmojaHack_Africa_Zindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries and Load Dataset**

In [ ]:
df_test='/content/drive/MyDrive/Test.csv'
df_train='/content/drive/MyDrive/Train.csv'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_test=pd.read_csv(df_test)
df_train=pd.read_csv(df_train)

In [ ]:
'concat train and test dataset'
df=pd.concat([df_train,df_test])

# **Data Pre-processing**

In [ ]:
"Label Encode categorical variable country"
country=df['country'].unique()
num=range(df['country'].nunique())
country_dict={}
for a,b in zip(country, num):
    df.replace(a,b, inplace=True)
    country_dict[a]=b



In [ ]:
"Label Encode categorical variable region"
region=df['region'].unique()
num=range(df['region'].nunique())
region_dict={}
for a,b in zip(region, num):
    df.replace(a,b, inplace=True)
    region_dict[a]=b

In [ ]:
'column Q7 has about 80% missing values, hence dropped'
df.drop('Q7', inplace=True, axis=1)

In [ ]:
'Split train and test Dataset'
df_train=df.iloc[:104867]
df_test=df.iloc[104867:]

In [ ]:
df_X=df_train.drop(['target', 'ID'], axis=1)
df_Y=df_train[['target']]

In [ ]:
'Fill missing values with mean'
for i in ['region','age', 'Q2','Q3','Q4','Q5','Q8','Q9','Q19', 'Q20','Q23','Q26','Q27','Q28', 'owns_mobile']:
      df_X.replace(np.nan, df_X[i].mean(),inplace=True)
      df_test.replace(np.nan, df_test[i].mean(),inplace=True)

# **Modelling Preparation**

In [ ]:
!pip install lightgbm
!pip install catboost



from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [ ]:
from sklearn import metrics
catboost=CatBoostClassifier()
lightgbm=LGBMClassifier()

In [ ]:
from sklearn.ensemble import VotingClassifier
vc=VotingClassifier(estimators=[('lg',lightgbm), ('cat', catboost)],
voting='soft', weights=[0.1, 0.35] )

## **Modelling and Prediction**

In [ ]:
df_ID=df_test[['ID']]

In [ ]:
df_test.drop(['ID','target'], axis=1, inplace=True)

In [ ]:
'stratified kfolding with voting classifier'
from sklearn.model_selection import StratifiedKFold
predictions=list()
kf=StratifiedKFold(n_splits=25)
for Train, Test in kf.split(df_X,df_Y):
    x_train,x_test,y_train,y_test=df_X.iloc[Train], df_X.iloc[Test], df_Y.iloc[Train], df_Y.iloc[Test]
    vc.fit(x_train, y_train)
    prediction=vc.predict_proba(df_test)[:,1]
    predictions.append(prediction)

In [ ]:
sub=np.mean(predictions, axis=0)

In [ ]:
pred=pd.DataFrame(sub)

In [ ]:
dfs=pd.concat([df_ID, pred], axis=1)

In [ ]:
from google.colab import files
dfs.to_csv('./submissionfile16.csv', sep = ',', index = False)

In [ ]:
files.download('./submissionfile16.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# The above modelling method gives an accuracy of 80% on the test set and 46th position of 463 participants on the Zindi LB.